# **VERSION 7**

---



##**A) pre-work segment**

###**1. download and install modules**

In [1]:
!pip install ConfigParser

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\backports\\__init__.py'
Consider using the `--user` option or check the permissions.



In [ ]:
#pip install googlemaps
#github!
#github2!
#github3

###**2. mount drive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

###**3. import needed classes and methods**

In [4]:
import configparser
from textblob import TextBlob
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import spacy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
#import googlemaps
from datetime import date
from datetime import timedelta
from bs4 import BeautifulSoup
import requests
#import sys
#import os

ModuleNotFoundError: No module named 'regex'

###**4. download from imported modules**

In [8]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

###**5. connect & authenticate**

In [13]:
!ls '/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties'

'/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties'


In [14]:
config=configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties')
print(config.sections())

['twitter', 'gmaps']


In [15]:
consumerKey=config.get('twitter','consumerKey')
consumerSecret=config.get('twitter','consumerSecret')
accessToken=config.get('twitter','accessToken')
accessTokenSecret=config.get('twitter','accessTokenSecret')

In [16]:
#print(consumerKey,"\n",consumerSecret,"\n",accessToken,"\n",accessTokenSecret)

In [18]:
auth=tweepy.OAuthHandler(consumer_key=consumerKey,consumer_secret=consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [19]:
#gmaps=googlemaps.Client(key=config.get('gmaps','gApiKey'))

##**B) methods for work segment**

###**1. send tweet**

In [20]:
#while True:
#  api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [21]:
#tw=tweepy.Cursor(api.search, q="covid"+"-filter:retweets",lang="English").items(100)
#for i in tw:
#  print([i.text,i.user.screen_name,i.user.location,i.geo])

In [22]:
def sendTweet():
  tweet=input("enter your tweet: ")
  api.update_status(tweet)

###**2. create dataset with specific keyword**

In [23]:
def searchTweetWithKeyword():

  covid_list=[]


  searchTerm,noOfRecords=input("enter keyword and max no. of records needed:").split()
  noOfRecords=int(noOfRecords)

  #a_tweets=tweepy.Cursor(api.search, q=searchTerm+"-filter:retweets",lang="English").items(noOfRecords)
  a_tweets=tweepy.Cursor(api.search, q=searchTerm+"-filter:retweets",lang="en").items(noOfRecords)

  for i in a_tweets:
    covid_list.append([i.text,i.user.screen_name,i.user.location,i.geo])


  covid_df=pd.DataFrame(covid_list,columns=["tweets","user_name","location","geo"])
  totalNoOfRecords=len(covid_df)

  print("total no. of records: ",totalNoOfRecords)
  print("records: ")
  display(covid_df)

  return covid_df,totalNoOfRecords

###**3. check tweet sentiments using TextBlob**

In [24]:
def percentage(part,whole):
  return part*100/whole


In [25]:
def checkSentimentTextBlob(covid_df,noOfRecords):
  positive=0
  negative=0
  neutral=0
  polarity=0

  for i in range(noOfRecords):
    textblob_text=TextBlob(covid_df.iloc[i,0])
    value=textblob_text.sentiment.polarity
    polarity+=value
    
    if value<0:
        negative+=1
    elif value==0:
        neutral+=1
    elif value>0:
        positive+=1

  print("raw numbers:\npositive=",positive,"\nnegative=",negative,"\nneutral=",neutral,"\npolarity=",polarity)
  
  positive=format(percentage(positive,noOfRecords),'.2f')
  negative=format(percentage(negative,noOfRecords),'.2f')
  neutral=format(percentage(neutral,noOfRecords),'.2f')
  polarity=format(percentage(polarity,noOfRecords),'.2f')

  print("percentage:\npositive=",positive,"\nnegative=",negative,"\nneutral=",neutral,"\npolarity=",polarity)

###**4. count tweet from each location (city)**

In [26]:
def tweetLocations(covid_df,noOfRecords):
  
  display(covid_df['location'].value_counts())

###**5. clean tweets**

In [27]:
def cleanTweets(covid_df,noOfRecords):
  for i in range(noOfRecords):
    text=covid_df.iloc[i,0]
    
    text=re.sub("@[\w]*","",text)
    text=re.sub("RT @[\w]*:","",text)
    text=re.sub("https?://[a-zA-Z0-9./]*","",text)
    text=re.sub("\n","",text)
    text=re.sub("#","",text)
    text=re.sub("[`~!@#$%^&*()+\[\]{}\\:;'\",=<>?/.|-]|[_]","",text)
    text= re.sub("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", "",text)

    covid_df.iloc[i,0]=text
  
  display(covid_df)
  return covid_df,noOfRecords

###**6. convert dataframe to csv**

In [28]:
def toCSV(covid_df):
  covid_df.to_csv('covid_csv.csv')

###**7. show entites in text**

In [29]:
def showEntities(covid_df,noOfRecords):
  nlp=spacy.load('en_core_web_sm')

  covid_df['entities']=covid_df['tweets'].apply(lambda tweet: [(ent.text,ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(tweet).ents])
  
  display(covid_df)
  return covid_df,noOfRecords

###**8. check tweet sentiments using Sentiment Intensity Analyzer**

In [30]:
def tweetSentimentIntensityAnalyzer(covid_df,noOfRecords):
  sid=SentimentIntensityAnalyzer()
  covid_df['sentiment']=covid_df['tweets'].apply(lambda tweet: sid.polarity_scores(tweet))
  display(covid_df)
  return covid_df,noOfRecords 

###**9. categorizing by location using googlemaps api**

In [31]:
#not working till we set payment details 

In [32]:
def getCountry(location):
  result=gmaps.geocode(location)
  result=(result[0]['formatted_address'].split(','))[-1].strip()
  return result

In [33]:
def tweetLocationsUsingGmaps(covid_df,noOfRecords):
  covid_df['country']=covid_df['location'].apply(lambda location: "" if(not location.strip() ) else getCountry(location))

  print(covid_df['country'].value_counts())

  return covid_df,noOfRecords


###**10. filter tweets in dataset by keywords**

In [34]:
def search(keyword,tweets):
  temp=[]
  for t in tweets:
    #print(type(t))
    #print(t)
    #print(re.search(keyword,t))
    if re.search(keyword.lower(),t.lower()):
      temp.append(True)
    else:
      temp.append(False)
  return temp

In [35]:
def filterDataSet(covid_df,noOfRecords):
  
  while True:
    searchTerm=input("enter keyword:")
    #covid_df=covid_df[ lambda covid_df.tweets,searchTerm: return re.search(keyword,text) ]
    ncovid_df=covid_df[ search(searchTerm,covid_df.tweets) ]
    display(ncovid_df)
    ch1=input("do you want to keep the changes?\n y or n:")
    if  (ch1=='y' or ch1=='Y'):
      covid_df=ncovid_df
      

    ch2=input("do you want to search for another keyword from within your dataset?\n y or n:")
    if not (ch2=='y' or ch2=='Y'):
      break

  print("FINAL DATASET:")
  totalNoOfRecords=len(covid_df)
  print("no. of records:",totalNoOfRecords)
  print("records:")
  display(covid_df)
  return covid_df,totalNoOfRecords

###**11. filter location in dataset by keywords**

In [36]:
def searchLoc(keyword,loc):
  temp=[]
  for t in loc:
    #print(type(t))
    #print(t)
    #print(re.search(keyword,t))
    if re.search(keyword.lower(),t.lower()):
      temp.append(True)
    else:
      temp.append(False)
  return temp

In [37]:
def filterDataSetUsingLoc(covid_df,noOfRecords):
  
  while True:
    searchTerm=input("enter location as keyword:")
    #covid_df=covid_df[ lambda covid_df.tweets,searchTerm: return re.search(keyword,text) ]
    ncovid_df=covid_df[ search(searchTerm,covid_df.location) ]
    display(ncovid_df)
    ch1=input("do you want to keep the changes?\n y or n:")
    if  (ch1=='y' or ch1=='Y'):
      covid_df=ncovid_df
      

    ch2=input("do you want to search for another keyword from within your dataset?\n y or n:")
    if not (ch2=='y' or ch2=='Y'):
      break
  print("FINAL DATASET:")
  totalNoOfRecords=len(covid_df)
  print("no. of records:",totalNoOfRecords)
  print("records:")
  display(covid_df)
  return covid_df,totalNoOfRecords

###**12. sentiment related to medical and other facilities related to covid state wise for India or city wise for some cities**


In [38]:
def get_wordnet_pos(treebank_tag):

  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [39]:
def postag(tokens):
  pos=nltk.pos_tag(tokens)
  print("pos:",pos)
  res=[]
  lemmatizer = WordNetLemmatizer()
  for i in pos:
    res.append(lemmatizer.lemmatize(i[0], pos=get_wordnet_pos(i[1]) ) )
  return res

In [40]:
def customSolution():
  covid,noOfRecords=searchTweetWithKeyword()
  covid_df=covid.copy()
  covid_df,noOfRecords=cleanTweets(covid_df,noOfRecords)


  stop_words = set(stopwords.words('english')) 
  ftokens_list=[]
  for i in range(noOfRecords):
    tokens=word_tokenize(covid_df.iloc[i,0].lower())
    filtered_tokens = [w  for w in tokens if not w in stop_words]
    ftokens_list.append(filtered_tokens)
  covid_df['ftokens']=pd.Series(ftokens_list)

  display(covid_df)
  covid_df['lemma']=covid_df['ftokens'].apply(lambda ftoken: postag(ftoken) )
  display(covid_df)


  noOfRecords=covid_df.shape[0]
  medic_terms=['medical','hospital','hospital bed','spread','outbreak','help','need','reinforcement','affected','discharge','tested','death']
  medic_terms=postag(medic_terms)
  print(medic_terms)

  # for i in range(noOfRecords):
  #   display(covid_df[ medic_lemmas is in for l in covid_df.loc[i,'lemma'] ])
  temp=[]
  for i in range(noOfRecords):
    flag=0
    for j in medic_terms:
      for k in covid_df.loc[i,'lemma']:
        if j==k:
          temp.append(True)
          flag=1
          break
      if flag==1:
        break
    if flag==0:
      temp.append(False)

  xcovid=covid_df[temp].copy()
  xcovid.reset_index(inplace=True)
  xcovid.drop('index',axis=1,inplace=True)
  display(xcovid)
  noOfRecords=xcovid.shape[0]


  positive=[]
  neutral=[]
  negative=[]

  for i in range(noOfRecords):
      textblob_text=TextBlob(xcovid.iloc[i,0])
      value=textblob_text.sentiment.polarity
      
      
      if value<0:
          positive.append(0)
          neutral.append(0)
          negative.append(1)

      elif value==0:
          positive.append(0)
          neutral.append(1)
          negative.append(0)

      elif value>0:
          positive.append(1)
          neutral.append(0)
          negative.append(0)

  xcovid['positive']=pd.Series(positive)
  xcovid['neutral']=pd.Series(neutral)
  xcovid['negative']=pd.Series(negative)

  display(xcovid)

  group=pd.DataFrame( )
  group['positive']=xcovid.groupby('location').positive.sum()
  group['neutral']=xcovid.groupby('location').neutral.sum()
  group['negative']=xcovid.groupby('location').negative.sum()
  display(group)

  
  covid.to_csv('covid.csv')
  covid_df.to_csv('fcovid.csv')
  xcovid.to_csv('xcovid.csv')
  group.to_csv('sen.csv')
  return 

###**13. webcrawl and append to dataset**

In [41]:
def webcrawl():
  pass


In [42]:
today=date.today()-timedelta(1)
d=today.strftime("%m-%d-%y")
print("date =",d)

cnn_url="https://edition.cnn.com/world/live-news/coronavirus-pandemic-{}-intl/index.html".format(d)
print(cnn_url)


date = 09-10-20
https://edition.cnn.com/world/live-news/coronavirus-pandemic-09-10-20-intl/index.html


In [43]:
html=requests.get(cnn_url).text
display(html)
print("*******************************************************************")
soup=BeautifulSoup(html)
display(soup)
print(soup.title)

Output hidden; open in https://colab.research.google.com to view.

##**C) main method segment**

In [44]:
  covid_df=None
  noOfRecords=None

In [45]:
def main(covid_df,noOfRecords):


  print(
  "press 0 to display dataframe\n"
  "press 1 to send tweet\n"
  "press 2 to find tweets with keyword\n"
  "press 3 to check sentiments using TextBlob\n"
  "press 4 to categorize and count by location (city)\n"
  "press 5 to clean tweets\n"
  "press 6 to covert dataframe into csv\n"
  "press 7 to show entities in text\n"
  "press 8 to check tweet sentiments using Sentiment Intensity Analyzer\n"
  "press 9 to categorize and count by location (country) using googlemaps api\n"
  "press 10 to filter tweets in dataset by keywords"
  "press 11 to find sentiment related to medical and other facilities related to covid state wise for India or city wise for some cities"
  "press 12 to webcrawl and append to dataset"
  "press 100 to exit\n")

  while True:
    ch=int(input("choose: "))
    if ch==0:
        display(covid_df)
    elif ch==1:
      sendTweet()
    elif ch==2:
      covid_df,noOfRecords=searchTweetWithKeyword()
    elif ch==3:
      checkSentimentTextBlob(covid_df,noOfRecords)
    elif ch==4:
      tweetLocations(covid_df,noOfRecords)
    elif ch==5:
      covid_df,noOfRecords=cleanTweets(covid_df,noOfRecords)
    elif ch==6:
      toCSV(covid_df)
    elif ch==7:
      covid_df,noOfRecords=showEntities(covid_df,noOfRecords)
    elif ch==8:
      covid_df,noOfRecords=tweetSentimentIntensityAnalyzer(covid_df,noOfRecords)
    elif ch==9:
      covid_df,noOfRecords=tweetLocationsUsingGmaps(covid_df,noOfRecords)
    elif ch==10:
      covid_df,noOfRecords=filterDataSet(covid_df,noOfRecords)
    elif ch==11:
      covid_df,noOfRecords=filterDataSetUsingLoc(covid_df,noOfRecords)
    elif ch==12:
      customSolution()
    elif ch==13:
      webcrawl()
    elif ch==100:
      break
    else:
      print("INVALID CHOICE !")

In [46]:
if __name__=="__main__":
  covid_df,noOfRecords=searchTweetWithKeyword()
  main(covid_df,noOfRecords)

enter keyword and max no. of records needed:asd 1
total no. of records:  1
records: 


,tweets,user_name,location,geo
0,"@MrsJacenSolo ""it's not a door. it's a doornt""",asd_riptide,The Age of Sin,None


press 0 to display dataframe
press 1 to send tweet
press 2 to find tweets with keyword
press 3 to check sentiments using TextBlob
press 4 to categorize and count by location (city)
press 5 to clean tweets
press 6 to covert dataframe into csv
press 7 to show entities in text
press 8 to check tweet sentiments using Sentiment Intensity Analyzer
press 9 to categorize and count by location (country) using googlemaps api
press 10 to filter tweets in dataset by keywordspress 11 to find sentiment related to medical and other facilities related to covid state wise for India or city wise for some citiespress 12 to webcrawl and append to datasetpress 100 to exit

choose: 12
enter keyword and max no. of records needed:covid 200
total no. of records:  200
records: 


,tweets,user_name,location,geo
0,@GMA Lol y’all still talking bout something th...,alotofthesame,"Los Angeles, CA",None
1,Covid-19 is worse than 9/11.\n\nDo you know that?,jakebritt00,"Newport, NH",None
2,A great article from our weekly selection. Afr...,IHSA_studies,"The Hague, The Netherlands",None
3,@BDUTT Community classes/ mid day meal distrib...,BhatNay68453245,Srinagar,None
4,I had plans to visit a friend today. A single ...,CAontu,Ireland,None
...,...,...,...,...
195,Sick to death over feeling guilty over stinkin...,ColferCat,New York,None
196,@dougmillsnyt @realDonaldTrump Not only is thi...,WeissA327,,None
197,"Easing of lockdown, means the economy inevitab...",nmcinroy,here and there,None
198,Adilabad: The only COVID-19 hospital located o...,AakeruNews,,None


,tweets,user_name,location,geo
0,Lol y’all still talking bout something that h...,alotofthesame,"Los Angeles, CA",None
1,Covid19 is worse than 911Do you know that,jakebritt00,"Newport, NH",None
2,A great article from our weekly selection Afri...,IHSA_studies,"The Hague, The Netherlands",None
3,Community classes mid day meal distribution c...,BhatNay68453245,Srinagar,None
4,I had plans to visit a friend today A single f...,CAontu,Ireland,None
...,...,...,...,...
195,Sick to death over feeling guilty over stinkin...,ColferCat,New York,None
196,Not only is this a covid nightmare it can’t ...,WeissA327,,None
197,Easing of lockdown means the economy inevitabl...,nmcinroy,here and there,None
198,Adilabad The only COVID19 hospital located on ...,AakeruNews,,None


...

pos: [('medical', 'JJ'), ('hospital', 'NN'), ('hospital bed', 'VBD'), ('spread', 'JJ'), ('outbreak', 'NN'), ('help', 'NN'), ('need', 'VB'), ('reinforcement', 'NN'), ('affected', 'JJ'), ('discharge', 'NN'), ('tested', 'VBD'), ('death', 'NN')]
['medical', 'hospital', 'hospital bed', 'spread', 'outbreak', 'help', 'need', 'reinforcement', 'affected', 'discharge', 'test', 'death']


,tweets,user_name,location,geo,ftokens,lemma
0,Lol y’all still talking bout something that h...,alotofthesame,"Los Angeles, CA",None,"[lol, ’, still, talking, bout, something, happ...","[lol, ’, still, talk, bout, something, happen,..."
1,Total COVID19 tests performed in Taiwan 181922...,COVID19_bot,,None,"[total, covid19, tests, performed, taiwan, 181...","[total, covid19, test, perform, taiwan, 181922..."
2,Indias coronavirus tally crossed the 45 millio...,WIONews,India,None,"[indias, coronavirus, tally, crossed, 45, mill...","[indias, coronavirus, tally, cross, 45, millio..."
3,What constitutes a number being counted in ...,ohyeahhii,,None,"[constitutes, number, counted, covid, death, t...","[constitutes, number, count, covid, death, tol..."
4,Johnson playing Pandemic like an episode of Wh...,swizzle61,United Kingdom,None,"[johnson, playing, pandemic, like, episode, wh...","[johnson, play, pandemic, like, episode, whack..."
5,The US is rapidly approaching 200k deaths from...,Klaige,,None,"[us, rapidly, approaching, 200k, deaths, covid...","[u, rapidly, approach, 200k, death, covid19the..."
6,UK gov is paving the way to use mass vaccinati...,HowardSteen4,"Hesse, Germany",None,"[uk, gov, paving, way, use, mass, vaccination,...","[uk, gov, pave, way, use, mass, vaccination, u..."
7,If one trust encourages doctors not to put C...,readthinkwalk,Milton Keynes,None,"[one, trust, encourages, doctors, put, covid19...","[one, trust, encourage, doctor, put, covid19, ..."
8,Tested ve for covid thank you all for your wi...,IshantJaiswal9,𝕹𝖊𝖜 𝕯𝖊𝖑𝖍𝖎,None,"[tested, covid, thank, wishes]","[test, covid, thank, wish]"
9,Absolutely true Filipino nurses in Hawaii are ...,canlas_gu,Hawaii,None,"[absolutely, true, filipino, nurses, hawaii, 5...","[absolutely, true, filipino, nurse, hawaii, 50..."


,tweets,user_name,location,geo,ftokens,lemma,positive,neutral,negative
0,Lol y’all still talking bout something that h...,alotofthesame,"Los Angeles, CA",None,"[lol, ’, still, talking, bout, something, happ...","[lol, ’, still, talk, bout, something, happen,...",1,0,0
1,Total COVID19 tests performed in Taiwan 181922...,COVID19_bot,,None,"[total, covid19, tests, performed, taiwan, 181...","[total, covid19, test, perform, taiwan, 181922...",0,1,0
2,Indias coronavirus tally crossed the 45 millio...,WIONews,India,None,"[indias, coronavirus, tally, crossed, 45, mill...","[indias, coronavirus, tally, cross, 45, millio...",1,0,0
3,What constitutes a number being counted in ...,ohyeahhii,,None,"[constitutes, number, counted, covid, death, t...","[constitutes, number, count, covid, death, tol...",0,1,0
4,Johnson playing Pandemic like an episode of Wh...,swizzle61,United Kingdom,None,"[johnson, playing, pandemic, like, episode, wh...","[johnson, play, pandemic, like, episode, whack...",0,1,0
5,The US is rapidly approaching 200k deaths from...,Klaige,,None,"[us, rapidly, approaching, 200k, deaths, covid...","[u, rapidly, approach, 200k, death, covid19the...",0,0,1
6,UK gov is paving the way to use mass vaccinati...,HowardSteen4,"Hesse, Germany",None,"[uk, gov, paving, way, use, mass, vaccination,...","[uk, gov, pave, way, use, mass, vaccination, u...",0,1,0
7,If one trust encourages doctors not to put C...,readthinkwalk,Milton Keynes,None,"[one, trust, encourages, doctors, put, covid19...","[one, trust, encourage, doctor, put, covid19, ...",0,1,0
8,Tested ve for covid thank you all for your wi...,IshantJaiswal9,𝕹𝖊𝖜 𝕯𝖊𝖑𝖍𝖎,None,"[tested, covid, thank, wishes]","[test, covid, thank, wish]",0,1,0
9,Absolutely true Filipino nurses in Hawaii are ...,canlas_gu,Hawaii,None,"[absolutely, true, filipino, nurses, hawaii, 5...","[absolutely, true, filipino, nurse, hawaii, 50...",1,0,0


,positive,neutral,negative
location,,,
,0,6,2
250,0,1,0
Australia,1,0,0
Brisbane-Tokyo-San Francisco,0,1,0
Devon,0,0,1
Hawaii,1,0,0
"Hesse, Germany",0,1,0
India,1,0,0
"Los Angeles, CA",1,0,0


choose: 100




---


# **END OF CODE, ROUGH WORK BEGINS:**


---



In [ ]:
print(__name__)

__main__


In [ ]:
def create_df():
  dfr=pd.DataFrame([['qwerty','be','mumbai, india','d','e'],['uiop','be','canada',None,'e'],['wasd','bi','dallas tx','d','e'],['fghjklzxc','b','european union','d','e']],columns=['A','B','C','D','E'])
  display("records:",dfr)
  return dfr

In [ ]:
dfr=create_df()

'records:'

,A,B,C,D,E
0,qwerty,be,"mumbai, india",d,e
1,uiop,be,canada,None,e
2,wasd,bi,dallas tx,d,e
3,fghjklzxc,b,european union,d,e


In [ ]:
dfr1=pd.DataFrame([['zxcasdedc','bo','kolkata, india','de','ei'],['uiop','be','orissa','dio','eu']],columns=['A','B','C','D','E'])
dfr=dfr.append(dfr1,ignore_index=True)
dfr.drop_duplicates(inplace=True)
dfr

#df.iloc[:,[1,3]]

,A,B,C,D,E
0,qwerty,be,"mumbai, india",d,e
1,uiop,be,canada,None,e
2,wasd,bi,dallas tx,d,e
3,fghjklzxc,b,european union,d,e
4,zxcasdedc,bo,"kolkata, india",de,ei
5,uiop,be,orissa,dio,eu


In [ ]:
dfr.iloc[3,2]

'european union'

In [ ]:
dfr['A']

0       qwerty
1         uiop
2         wasd
3    fghjklzxc
4    zxcasdedc
5         uiop
Name: A, dtype: object

In [ ]:
listo=[]
for i in range(len(dfr)):
  listo.append(dfr.iloc[i,0])

print(listo)

['qwerty', 'uiop', 'wasd', 'fghjklzxc', 'zxcasdedc', 'uiop']


In [ ]:
dfr['B'].value_counts()

be    3
bo    1
b     1
bi    1
Name: B, dtype: int64

In [ ]:
nlp=spacy.load('en_core_web_sm')
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
print(type(text))


for ent in nlp(text).ents:
  print(ent.text,ent.label_)

<class 'str'>
Sebastian NORP
Google ORG
2007 DATE
American NORP
Recode ORG
earlier this week DATE


In [ ]:
dfr['A'].apply(lambda x: x+': test')

0       qwerty: test
1         uiop: test
2         wasd: test
3    fghjklzxc: test
4    zxcasdedc: test
5         uiop: test
Name: A, dtype: object

In [ ]:
dfr['A'].apply(lambda x: [len(x) for i in range(len(x))])

0             [6, 6, 6, 6, 6, 6]
1                   [4, 4, 4, 4]
2                   [4, 4, 4, 4]
3    [9, 9, 9, 9, 9, 9, 9, 9, 9]
4    [9, 9, 9, 9, 9, 9, 9, 9, 9]
5                   [4, 4, 4, 4]
Name: A, dtype: object

In [ ]:
dfr['A'].apply(lambda x: ["e" if i%2==0 else "o"  for i in range(len(x))])

0             [e, o, e, o, e, o]
1                   [e, o, e, o]
2                   [e, o, e, o]
3    [e, o, e, o, e, o, e, o, e]
4    [e, o, e, o, e, o, e, o, e]
5                   [e, o, e, o]
Name: A, dtype: object

In [ ]:
dfr['e/o 1']=dfr['A'].apply(lambda x: ["e" if i%2==0 else "o"  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]"
1,uiop,be,canada,None,e,"[e, o, e, o]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]"
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]"
5,uiop,be,orissa,dio,eu,"[e, o, e, o]"


In [ ]:
dfr['e/o 2']=dfr['A'].apply(lambda x: [(i,"e") if i%2==0 else (i,"o")  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1,e/o 2
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o)]"
1,uiop,be,canada,None,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o..."
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o..."
5,uiop,be,orissa,dio,eu,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"


In [ ]:
dfr['e/o 3']=dfr['A'].apply(lambda x: [(i,"e") if i%2==0 else ""  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1,e/o 2,e/o 3
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o)]","[(0, e), , (2, e), , (4, e), ]"
1,uiop,be,canada,None,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o...","[(0, e), , (2, e), , (4, e), , (6, e), , (8, e)]"
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o...","[(0, e), , (2, e), , (4, e), , (6, e), , (8, e)]"
5,uiop,be,orissa,dio,eu,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"


In [ ]:
dfr.to_csv('df.csv')

!ls

covid_csv.csv  df.csv  drive  sample_data


In [ ]:
# gmaps=googlemaps.Client(key=config.get('gmaps','gApiKey'))
# geocode_result=gmaps.geocode(df.iloc[2,2])

# print(geocode_result)
# print(geocode_result[0])
# print(geocode_result[0]['formatted_address'])
# print(geocode_result[0]['formatted_address'].split(',')[-1])
# print(geocode_result[0]['formatted_address'].split(',')[-1].strip())


In [ ]:
trial="  asd  "
print(not trial.strip())
print(not( not trial.strip()))

trial="    "
print(not trial.strip())
print(not( not trial.strip()))

False
True
True
False


In [ ]:
class StreamListner(tweepy.StreamListner);
def on_status(self,status):
  print(status)
def on_error(self,error,    ):
  print(error)
def on_data(self,data):


SyntaxError: ignored

In [ ]:
#covid_confirmed_df=pd.read_csv("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
print(u"\U0001F600")

😀


In [ ]:
text = u'This is a smiley face \U0001f602'
print(text) # with emoji

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

print(deEmojify(text))

This is a smiley face 😂
This is a smiley face 


In [ ]:
text="looking ok forward to your show is basically an exciting adventure"
tokens=word_tokenize(text)
postag(tokens)

In [ ]:
qwertyuiop="asdfghjkl"
print(qwertyuiop[2:5])

dfg


In [ ]:
a=23.8784632954251
print('this is {}'.format(a) )

this is 23.8784632954251


abcdcdc
cdc
cdc
cdc
2


4